# Introduction to Web Scraping

Web scraping is the process of extracting data from websites. It allows you to collect structured data from the web, which can be used for analysis, visualization, or other applications in `data science`. In this tutorial, we'll cover the basics of web scraping using Python and the BeautifulSoup library.
Also we will look at ethical considerations when scraping websites, along with overview of API's and how they can be used to extract data from websites.

### What is Web Scraping?
Web scraping involves the following steps:
1. Sending a request to a website.
2. Retrieving the content of the webpage.
3. Parsing the content to extract specific information.
4. Saving the extracted data for further use.

### Ethical Considerations
Before scraping, ensure the website permits it. Check the website's `robots.txt` file and adhere to its guidelines.

# Setting Up the Environment
We'll use the following libraries in this tutorial:

- `requests`: To send HTTP requests to fetch web pages.
- `BeautifulSoup`: To parse and extract data from HTML content.

Run the following cell to install the required libraries.

In [ ]:
! pip install requests beautifulsoup4

# Step 1: Fetching the Webpage

We'll begin by sending a GET request to the demo website and fetching its HTML content. The `requests` library is used for this purpose.

In [9]:
import requests

# URL of the demo website
url = "https://cpatel321.github.io/webscrapping-tutorial/text.html"

response = requests.get(url)

# Check the status code
if response.status_code == 200:
    print("Page fetched successfully!")
    print(response.text[:])  # Print the first 500 characters of the HTML
else:
    print("Failed to fetch the page. Status code:", response.status_code)

Page fetched successfully!
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="style.css">
    <title>DataLand - Text Content</title>
</head>
<body>
    <header>
        <h1>Random Text Content</h1>
        <nav>
            <ul>
                <li><a href="index.html">Home</a></li>
                <li><a href="table.html">Data Tables</a></li>
                <li><a href="text.html">Text Content</a></li>
                <li><a href="figures.html">Figures & Statistics</a></li>
            </ul>
        </nav>
    </header>
    <section>
        <article>
            <h2>Article 1: Random Facts</h2>
            <p>Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.</p>
            <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
 

# Step 2: Parsing the Webpage
We'll use BeautifulSoup to parse the HTML content and extract meaningful data. In this case, we aim to retrieve the text content of all articles from the demo webpage.

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser') # parse the HTML content using BeautifulSoup

articles = soup.find_all('article') # takes html tags as args, returns a list of all tags

# Extract and print the text from each article
for idx, article in enumerate(articles, start=1):
    heading = article.find('h2').get_text(strip=True) # stripe justifies the text by removing leading and trailing whitespaces
    content = article.find_all('p')
    print(f"Match ID {idx}: {heading}")
    for para in content:
        print(para.get_text(strip=True))
    print("\n")

Match ID 1: Article 1: Random Facts
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.
Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.


Match ID 2: Article 2: More Random Facts
Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.
Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.




# Step 3: Saving Data
Extracted data can be saved to a file for further use. In this example, we'll save the article data to a text file.

In [13]:
# saving articles to a text file
with open("articles.txt", "w", encoding="utf-8") as f:
    for idx, article in enumerate(articles, start=1):
        heading = article.find('h2').get_text(strip=True)
        content = article.find_all('p')
        f.write(f"Article {idx}: {heading}\n")
        for para in content:
            f.write(para.get_text(strip=True) + "\n")
        f.write("\n")
print("Articles saved to articles.txt")

Articles saved to articles.txt


# Step 4: Scraping Tables
If the webpage contains tables, they can be extracted and saved as CSV files for easier analysis.

In [24]:
import csv #for parsing the data into csv and other type conversions

#Here we will again send a request to a different webpage containing tables and extract the data from the tables
table_url = "https://cpatel321.github.io/webscrapping-tutorial/table.html"
soup_table = BeautifulSoup(requests.get(table_url).text, 'html.parser')

tables = soup_table.find_all('table') # table is the tag used by most of the tables we see on websites

if tables: # list is not empty
    table = tables[0] # picking the first table 
    # print table data 
    rows = table.find_all('tr')

    # Open a CSV file for writing
    with open("table_data.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f) 
        
        
        #write each row to the CSV file
        for row in rows:
            cells = row.find_all(['th', 'td'])
            writer.writerow([cell.get_text(strip=True) for cell in cells])
            print([cell.get_text(strip=True) for cell in cells])
    print("\nTable data saved to table_data.csv")
else:
    print("No tables found on the page.")

['ID', 'Name', 'Age', 'Country']
['1', 'John Doe', '30', 'USA']
['2', 'Jane Smith', '25', 'Canada']

Table data saved to table_data.csv


In [ ]:
import urljoin
image_url = "https://cpatel321.github.io/webscrapping-tutorial/figures.html"


# Parse the HTML
soup_image = BeautifulSoup(requests.get(image_url).text, 'html.parser')

# Find all images and their associated text
images = soup_image.find_all('img')
text_data = soup_image.find_all(['p', 'ul'])

# Create a CSV file to store the scraped data
with open('images_and_text.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Image URL", "Description"])  # Write the header

    # Loop through each image and corresponding text
    for image in images:
        img_url = image['src']
        # full url of image to dowload it
        img_url = urljoin(image_url, img_url)
        # saving the image in scrapped folder
        img_data = requests.get(img_url).content
        with open(f"image_{images.index(image)}.jpg", 'wb') as handler:
            handler.write(img_data)
        # img_alt = image['alt']
        
        # Find the associated description text (using next sibling or related paragraph)
        associated_text = ""
        for text in text_data:
            if text.find_parent('section'):
                associated_text = text.get_text(strip=True)
                break  # Stop at the first relevant description

        # Write the image URL and associated text to CSV
        writer.writerow([img_url, associated_text])

print("Data has been written to 'images_and_text.csv'.")

Data has been written to 'images_and_text.csv'.


# Final Notes

This tutorial demonstrated the basics of web scraping using Python. Remember to respect the terms of use of websites and avoid overloading servers with too many requests in a short time.